In [83]:
from ortools.linear_solver import pywraplp
import numpy as np

In [84]:
def modelo_master(M, R, Q, P, scores, m, r, q, p, N, model=1):
    """
    pre:
        ya están ordenados los chabones por score decreciente
    """
    
    K = len(scores)
    
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return None

    # Variables
    x = {}
    for i in range(K):
        x[i] = solver.BoolVar(f'x[{i}]')

    # Restricciones
    solver.Add(solver.Sum([x[i] for i in range(K)]) == N)
    solver.Add(solver.Sum([x[i] for i in M.nonzero()[0]]) >= m * N/100)
    solver.Add(solver.Sum([x[i] for i in R.nonzero()[0]]) >= r * N/100)
    solver.Add(solver.Sum([x[i] for i in Q.nonzero()[0]]) >= q * N/100)
    solver.Add(solver.Sum([x[i] for i in P.nonzero()[0]]) >= p * N/100)

    if model == 1:
        solver.Maximize(solver.Sum([scores[i] * x[i] for i in range(K)]))
    elif model == 2:
        solver.Minimize(solver.Sum([ i * x[i] for i in range(K)]))
    elif model ==3:
        y = solver.IntVar(0, K, 'y')

        for i in range(K):
            solver.Add(i * x[i] <= y)

        # Objective Function
        solver.Minimize(y + 0.0002 * solver.Sum([ i * x[i] for i in range(K)]))

    return solver, x

In [91]:
# Total
K = 100
N = 50
# Porcentajes de admisión
m = 30
r = 60
q = 80
p = 0

In [92]:
def generar_instancia(K, m_pob = 0.5, r_pob = 0.3, q_pob = 0.7, p_pob = 0.2):
    M = np.random.uniform(0, 1, K) < m_pob
    R = np.random.uniform(0, 1, K) < r_pob
    Q = np.random.uniform(0, 1, K) < q_pob
    P = np.random.uniform(0, 1, K) < p_pob
    scores = np.random.rand(K)*100
    scores = np.sort(scores)[::-1]
    return M, R, Q, P, scores

In [101]:
generadas = generar_instancia(K)

In [114]:
solver, x = modelo_master(*generadas, m, r, q, p, N, 3 )

In [115]:
print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print("Solution:")
    print("Objective value =", solver.Objective().Value())
    for t in range(K):
        if x[t].solution_value() != 0:
            print(f"x[{t}] =", x[t].solution_value())
else:
    print("The problem does not have an optimal solution.")

print("\nAdvanced usage:")
print(f"Problem solved in {solver.wall_time():d} milliseconds")
print(f"Problem solved in {solver.iterations():d} iterations")
print(f"Problem solved in {solver.nodes():d} branch-and-bound nodes")

Solving with SCIP 8.0.4 [LP solver: Glop 9.8]
Solution:
Objective value = 93.3546
x[0] = 1.0
x[1] = 1.0
x[2] = 1.0
x[3] = 1.0
x[4] = 1.0
x[5] = 1.0
x[6] = 1.0
x[7] = 1.0
x[8] = 1.0
x[9] = 1.0
x[10] = 1.0
x[11] = 1.0
x[12] = 1.0
x[13] = 1.0
x[14] = 1.0
x[15] = 1.0
x[16] = 1.0
x[17] = 1.0
x[18] = 1.0
x[19] = 1.0
x[20] = 1.0
x[21] = 1.0
x[22] = 1.0
x[23] = 1.0
x[24] = 1.0
x[25] = 1.0
x[26] = 1.0
x[27] = 1.0
x[28] = 1.0
x[32] = 1.0
x[33] = 1.0
x[38] = 1.0
x[39] = 1.0
x[43] = 1.0
x[50] = 1.0
x[54] = 1.0
x[63] = 1.0
x[65] = 1.0
x[66] = 1.0
x[70] = 1.0
x[72] = 1.0
x[76] = 1.0
x[78] = 1.0
x[79] = 1.0
x[80] = 1.0
x[82] = 1.0
x[83] = 1.0
x[85] = 1.0
x[86] = 1.0
x[93] = 1.0

Advanced usage:
Problem solved in 2835 milliseconds
Problem solved in 62 iterations
Problem solved in 1 branch-and-bound nodes
